In [1]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from scipy import signal
import numpy as np

In [2]:
cat_directory = 'space_apps_2024_seismic_detection/data/lunar/training/catalogs/'
cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
cat = pd.read_csv(cat_file)

In [3]:
row = cat.filename
row

0     xa.s12.00.mhz.1970-01-19HR00_evid00002
1     xa.s12.00.mhz.1970-03-25HR00_evid00003
2     xa.s12.00.mhz.1970-03-26HR00_evid00004
3     xa.s12.00.mhz.1970-04-25HR00_evid00006
4     xa.s12.00.mhz.1970-04-26HR00_evid00007
                       ...                  
71    xa.s12.00.mhz.1974-10-14HR00_evid00156
72    xa.s12.00.mhz.1975-04-12HR00_evid00191
73    xa.s12.00.mhz.1975-05-04HR00_evid00192
74    xa.s12.00.mhz.1975-06-24HR00_evid00196
75    xa.s12.00.mhz.1975-06-26HR00_evid00198
Name: filename, Length: 76, dtype: object

In [4]:
data_directory = './space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/'


In [5]:
def process_seismic_data(tr_data_filt, sampling_rate, nperseg=256):
    frequencies, times, spectrogram = signal.spectrogram(
        tr_data_filt, 
        fs=sampling_rate,
        nperseg=nperseg
    )
    
    # Filter frequencies <= 2 Hz
    freq_mask = frequencies <= 2
    frequencies = frequencies[freq_mask]
    spectrogram = spectrogram[freq_mask, :]
    
    # Get dimensions
    n_timepoints = spectrogram.shape[1]
    n_freq_bins = spectrogram.shape[0]
    
    # Create flattened feature vector
    X = np.zeros(n_timepoints * (n_freq_bins + 1))
    
    # For each time point in the spectrogram
    for i in range(n_timepoints):
        base_idx = i * (n_freq_bins + 1)
        
        # Add the velocity value
        start_idx = i * (nperseg // 2)
        end_idx = start_idx + nperseg
        if end_idx > len(tr_data_filt):
            end_idx = len(tr_data_filt)
        X[base_idx] = np.mean(tr_data_filt[start_idx:end_idx])
        
        # Add the frequency components
        X[base_idx+1:base_idx+n_freq_bins+1] = spectrogram[:, i]
    
    # Reshape X to be a single row
    X = X.reshape(1, -1)
    
    # Create feature names
    feature_names = []
    for t_idx in range(n_timepoints):
        feature_names.extend([f'time_{t_idx:.3f}s_velocity'] + 
                            [f'time_{t_idx:.3f}s_freq_{f:.2f}Hz' for f in frequencies])
    
    return X, feature_names, times, frequencies

In [6]:
ct = 0
for each in row:
    csv_file = f'{data_directory}{each}.csv'
    data_cat = pd.read_csv(csv_file)
    print(ct)
    ct += 1

    csv_times = np.array(data_cat['time_rel(sec)'].tolist())
    csv_data = np.array(data_cat['velocity(m/s)'].tolist())
    X, feature_names, times, frequencies = process_seismic_data(
        tr_data_filt,
        tr_filt.stats.sampling_rate
    )


0


NameError: name 'tr_data_filt' is not defined

In [ ]:
csv_data.shape

# DATA PROCESSING

In [1]:
from obspy.signal import filter
import numpy as np
import pandas as pd
from obspy import read
from scipy import signal
import os

def process_seismic_data(tr_data_filt, sampling_rate, nperseg=256):
    frequencies, times, spectrogram = signal.spectrogram(
        tr_data_filt, 
        fs=sampling_rate,
        nperseg=nperseg
    )
    
    # Filter frequencies <= 2 Hz
    freq_mask = frequencies <= 2
    frequencies = frequencies[freq_mask]
    spectrogram = spectrogram[freq_mask, :]
    
    # Get dimensions
    n_timepoints = spectrogram.shape[1]
    n_freq_bins = spectrogram.shape[0]
    
    # Create flattened feature vector
    X = np.zeros(n_timepoints * (n_freq_bins + 1))
    
    # For each time point in the spectrogram
    for i in range(n_timepoints):
        base_idx = i * (n_freq_bins + 1)
        
        # Add the velocity value
        start_idx = i * (nperseg // 2)
        end_idx = start_idx + nperseg
        if end_idx > len(tr_data_filt):
            end_idx = len(tr_data_filt)
        X[base_idx] = np.mean(tr_data_filt[start_idx:end_idx])
        
        # Add the frequency components
        X[base_idx+1:base_idx+n_freq_bins+1] = spectrogram[:, i]
    
    # Reshape X to be a single row
    X = X.reshape(1, -1)
    
    # Create feature names
    feature_names = []
    for t_idx in range(n_timepoints):
        feature_names.extend([f'time_{t_idx:.3f}s_velocity'] + 
                            [f'time_{t_idx:.3f}s_freq_{f:.2f}Hz' for f in frequencies])
    
    return X, feature_names, times, frequencies

# Set up paths
cat_directory = 'space_apps_2024_seismic_detection/data/lunar/training/catalogs/'
cat_file = os.path.join(cat_directory, 'apollo12_catalog_GradeA_final.csv')
data_directory = './space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/'

# Read the catalog
cat = pd.read_csv(cat_file)

# Initialize an empty list to store all processed data
all_processed_data = []

# Loop through each file in the catalog
for idx, row in cat.iterrows():
    filename = row['filename']
    csv_file = os.path.join(data_directory, f"{filename}.csv")
    
    try:
        # Read the CSV file
        data_cat = pd.read_csv(csv_file)
        
        # Extract time and velocity data
        csv_times = np.array(data_cat['time_rel(sec)'].tolist())
        csv_data = np.array(data_cat['velocity(m/s)'].tolist())
        
        # Apply a bandpass filter (you may need to adjust these frequencies)
        filtered_data = filter.bandpass(csv_data, freqmin=0.5, freqmax=1.0, df=1.0/np.mean(np.diff(csv_times)))
        
        # Process the seismic data
        X, feature_names, times, frequencies = process_seismic_data(filtered_data, 1.0/np.mean(np.diff(csv_times)))
        
        # Create a DataFrame for this file's processed data
        df = pd.DataFrame(X, columns=feature_names)
        
        # Add metadata columns
        df['filename'] = filename
        df['arrival_time'] = row['time_rel(sec)']
        
        # Append to the list of all processed data
        all_processed_data.append(df)
        
        print(f"Processed file {idx+1}/{len(cat)}: {filename}")
    
    except Exception as e:
        print(f"Error processing file {filename}: {str(e)}")

# Combine all processed data into a single DataFrame
final_df = pd.concat(all_processed_data, ignore_index=True)

# Save the final DataFrame to a CSV file
output_file = 'processed_seismic_data.csv'
final_df.to_csv(output_file, index=False)
print(f"Processed data saved to {output_file}")

Processed file 1/76: xa.s12.00.mhz.1970-01-19HR00_evid00002
Processed file 2/76: xa.s12.00.mhz.1970-03-25HR00_evid00003
Processed file 3/76: xa.s12.00.mhz.1970-03-26HR00_evid00004
Processed file 4/76: xa.s12.00.mhz.1970-04-25HR00_evid00006
Processed file 5/76: xa.s12.00.mhz.1970-04-26HR00_evid00007
Processed file 6/76: xa.s12.00.mhz.1970-06-15HR00_evid00008
Processed file 7/76: xa.s12.00.mhz.1970-06-26HR00_evid00009
Processed file 8/76: xa.s12.00.mhz.1970-07-20HR00_evid00010
Processed file 9/76: xa.s12.00.mhz.1970-07-20HR00_evid00011
Processed file 10/76: xa.s12.00.mhz.1970-09-26HR00_evid00013
Processed file 11/76: xa.s12.00.mhz.1970-10-24HR00_evid00014
Processed file 12/76: xa.s12.00.mhz.1970-11-12HR00_evid00015
Processed file 13/76: xa.s12.00.mhz.1970-12-11HR00_evid00017
Processed file 14/76: xa.s12.00.mhz.1970-12-27HR00_evid00019
Processed file 15/76: xa.s12.00.mhz.1970-12-31HR00_evid00021
Processed file 16/76: xa.s12.00.mhz.1971-01-15HR00_evid00022
Processed file 17/76: xa.s12.00.m